In [ ]:
#単一の CW レーザーを照射 ver.
using DelimitedFiles
using SharedArrays
using Random
using Distributed

core_num = 20 #number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "D:\\シミュレーション工学\\sim_data"

@everywhere begin
    using SharedArrays
    using Distributed

    #parameters
    Tave = 298 #room temperature
    r_p = 100e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 10e-3 #incident laser power(mW)
    ls_spot_x = 0 #x-position of the center of the laser spot
    ls_spot_y = 0 #y-position of the center of the laser spot
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    FWHM = 1.22 * w_l / NA #full width at half maximum of the gaussian beam
    g_w = FWHM * sqrt(log(2) / 2) #Gaussian width at half maximum of the gaussian beam
    p_num = 200 #number of particles
    t = 60 #calculate time
    n = 60000 #steps
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    #dynamic viscosity of water
    function eta(T)
        eta = 2.761 * exp(1713 / T) * 10 ^ (-6)
        return eta
    end
    #variance of the BM
    function sigma(T)
        sigma = sqrt((2 * kb * T * deltat) / (6 * pi * eta(T) * r_p))
        return sigma
    end
    #friction coefficient
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    #generate random numbers
    function box_muller(u_1, u_2)
        X = sqrt.(-2 .* log.(u_1)) .* cos.(2 * pi .* u_2)
        Y = sqrt.(-2 .* log.(u_1)) .* sin.(2 * pi .* u_2)
        return X, Y
    end
    #optical gradient force of x-y plane
    #Here, we defined a optical gradient force of x-direction fx. If you calcuate a optical gradient force of y-direction fy, write ot(y, x).
    function fot(U0, x, y, r)
        fot = -4 * x * U0 * (r ^ -2) * exp(-2 * (x ^ 2 + y ^ 2) * r ^ (-2))
        return fot
    end
    #intensity of the laser
    function Int(power, rl)
        I = power / (pi * rl ^ 2)
        return I
    end
    #polarizability of particles
    function pol(a, np, nm)
        pol = 4 * pi * r_p ^ 3 * ((np ^ 2 - nm ^ 2) / (np ^ 2 + 2 * nm ^ 2))
        return pol
    end
    #depth of potential
    function U0(power, rl, a, np, nm)
        U0 = (Int(power, rl) * pol(a, np, nm) * nm) / c
        return U0
    end

    #generate displacements of BM
    function BM(p_num, T)
        xr = SharedArray{Float64}(p_num, n)
        yr = SharedArray{Float64}(p_num, n)
        @sync @distributed for i in 1:n
            u1 = rand(p_num, 1)
            u2 = rand(p_num, 1)
            X, Y = box_muller(u1, u2)
            xr[:, i] = X[:, 1]
            yr[:, i] = Y[:, 1]
        end
        sigmab = sigma(T)
        deltaxb = sigmab .* xr
        deltayb = sigmab .* yr
        return deltaxb, deltayb
    end

    #calculate a particle behavior
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, g_w, np, nm)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        xb = deltaxb[j, :]
        yb = deltayb[j, :]
        gammaf = gamma(Tave, r_p)
        U0f = U0(ls_pow, g_w, r_p, np, nm)
        #Let's calculate a particle behavior
        for i in 1:n
            x_i[i+1] = x_i[i] + xb[i] + fot(U0f, x_i[i] - ls_spot_x, y_i[i] - ls_spot_y, g_w) * deltat / gammaf
            y_i[i+1] = y_i[i] + yb[i] + fot(U0f, y_i[i] - ls_spot_y, x_i[i] - ls_spot_x, g_w) * deltat / gammaf
        end
        return x_i, y_i
    end
    #define parallel calculations
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, g_w, np, nm)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, g_w, np, nm)
        end

        if output == "um"
            resultx = resultx .* 1e6
            resulty = resulty .* 1e6
        end
        cd("$(save_path)")
        writedlm("x_$(ls_pow)mW_$(r_p)nm_sili2.txt", resultx, "\t")
        writedlm("y_$(ls_pow)mW_$(r_p)nm_sili2.txt", resulty, "\t")
    end

    #main
    function main(p_num, n, Tave, r_p, ls_pow, g_w, np, nm, ran)
        #generate displacements of Brrownian motion
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        #generate initial positionas
        x0 = rand(p_num, 1) .* 2 * -ran * 1e-6 .+ ran * 1e-6
        y0 = rand(p_num, 1) .* 2 * -ran * 1e-6 .+ ran * 1e-6
        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, g_w, np, nm)
    end
end
@time main(p_num, n, Tave, r_p, ls_pow, g_w, np, nm, ran)

 14.980535 seconds (72.41 M allocations: 10.902 GiB, 2.59% gc time, 2 lock conflicts, 1.57% compilation time)
